<a href="https://colab.research.google.com/github/selinakwok/flatmate.ucl/blob/main/flatmate.ucl%20google%20colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
from random import randrange



#from pydrive.auth import GoogleAuth 
#from pydrive.drive import GoogleDrive
#from google.colab import auth
#from oauth2client.client import GoogleCredentials
#auth.authenticate_user()
#gauth = GoogleAuth()
#gauth.credentials = GoogleCredentials.get_application_default()
#drive = GoogleDrive(gauth)

#import logging
#logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

#file_list = drive.ListFile({'q': "'12Z4sk7nySNKBMg1MwypgO_y0L0roxivxcemEkyHBv60' in parents and trashed=false"}).GetList()
#for file1 in file_list:
#  print('title: %s, id: %s' % (file1['title'], file1['id']))


from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials

# setup
gc = gspread.authorize(GoogleCredentials.get_application_default())

# read data and put it in a dataframe
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/12Z4sk7nySNKBMg1MwypgO_y0L0roxivxcemEkyHBv60/edit#gid=0')
sheets = gsheets.worksheet('Sheet1').get_all_values()

In [ ]:
df = pd.DataFrame(sheets[1:], columns=sheets[0])

In [ ]:
Religion = ["Christianity", "Islam", "Hinduism", "Judaism", "N.A."]
Gender = ["Male", "Female", "N.A."]
Diet = ["Vegan", "Hallal", "Kosher", "N.A."]
Budget_weekly_max = ["200", "250", "300", "350", "400"]
Location = ["Camden", "Islington", "City of Westminster", "City of London", "Kensingon and Chelsea",  "N.A."]
Total_flatmates = ["2", "3", "4", "5", ">5"]
Bedtime = ["8pm-10pm","10pm-12am", "12am-2am", "N.A."]
Smoking = ["yes", "no", "N.A."]
Language = ["English", "Mandarin", "Spanish", "French", "N.A."]

In [ ]:
# Creating a list of all caracteristics and a dictionary mapping each caracteristic to its respective list of attributes
# Will be useful later to create loops 
caracteristics = ["Religion", "Gender", "Diet", "Budget_weekly_max", "Location", "Total_flatmates", "Bedtime", "Smoking", "Language"]
from_caracteristic_to_fields = {"Religion": Religion, "Gender": Gender, "Diet": Diet, "Budget_weekly_max": Budget_weekly_max, "Location": Location, "Total_flatmates": Total_flatmates, "Bedtime": Bedtime, "Smoking": Smoking, "Language": Language}

In [ ]:
# First loop is creating empty columns for each caracteristic
# Second loop inserts a random amount of values from the respective list of attributes
# The function choice() has no parameter to avoid repetition, we deal with this below
rng = np.random.default_rng()
for j in range(len(caracteristics)):
  c = caracteristics[j]
  df[c] = np.nan
  for i in range(len(df)):
    df[c].iloc[i]= rng.choice(from_caracteristic_to_fields[c], random.randint(1,len(from_caracteristic_to_fields[c])))

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
# del df['Bedtime'], del df['Location'], and del df['Budget_weekly_max'] were used to remove these columns from the df dataframe

In [ ]:
# First loop iterates through the columns of the dataframe, second loop trough the rows
# Both loops remove duplicate values of every list in the dataframe
for k in range(1, len(df.columns)):
  for l in range(len(df)):
    df.iat[l,k] = list(dict.fromkeys(df.iat[l,k]))

In [ ]:
df.head()

,Name,Religion,Gender,Diet,Total_flatmates,Smoking,Language
0,Ben,"[Islam, Christianity, Hinduism]","[Female, N.A.]","[Kosher, Hallal]",[4],[yes],"[N.A., Mandarin]"
1,Max,"[Islam, Christianity, Judaism]","[N.A., Male]","[Vegan, N.A., Hallal]","[>5, 4, 2, 5]",[N.A.],"[French, Spanish, Mandarin]"
2,Lapo,[Christianity],[Female],[Kosher],"[>5, 3, 4]","[yes, no]",[English]
3,Raphael,[Judaism],[Female],"[Kosher, Vegan]","[3, 5]",[N.A.],"[English, Spanish]"
4,Yu,"[Judaism, Hinduism, N.A.]","[N.A., Female]","[N.A., Kosher, Vegan]",[2],[no],"[French, N.A.]"


In [ ]:
# Start from scratch (That is, only the names column in a new copy of the dataframe)
df_personal = pd.DataFrame(sheets[1:], columns=sheets[0])

In [ ]:
# Creating the dataframe for personal information, again randomly picking one attribute per charachteristic
for j in range(len(caracteristics)):
  c = caracteristics[j]
  df_personal[c] = np.nan
  for i in range(len(df)):
    df_personal[c].iloc[i]= random.choice(from_caracteristic_to_fields[c])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_personal.head()

,Name,Religion,Gender,Diet,Budget_weekly_max,Location,Total_flatmates,Bedtime,Smoking,Language
0,Ben,N.A.,Male,Vegan,300,City of Westminster,3,N.A.,N.A.,N.A.
1,Max,N.A.,Male,N.A.,200,Camden,3,8pm-10pm,yes,French
2,Lapo,Judaism,Male,N.A.,350,N.A.,>5,12am-2am,no,Mandarin
3,Raphael,Judaism,Female,Hallal,350,N.A.,2,12am-2am,no,N.A.
4,Yu,Hinduism,N.A.,Hallal,350,Islington,4,10pm-12am,yes,French


In [ ]:
# Dropping the unnecessary columns for this part
df_personal_short = df_personal.drop(["Religion", "Gender", "Diet", "Total_flatmates", "Smoking", "Language"], axis=1)
df_personal_short.head()

,Name,Budget_weekly_max,Location,Bedtime
0,Ben,300,City of Westminster,N.A.
1,Max,200,Camden,8pm-10pm
2,Lapo,350,N.A.,12am-2am
3,Raphael,350,N.A.,12am-2am
4,Yu,350,Islington,10pm-12am


In [ ]:
# This function takes a name as an input and returns all the people (by name) who have the same budget, location, and bedtime preferences
def find_compatible(index):
  name = df_personal_short.iloc[index,0]
  weekly_max = df_personal_short.iloc[index,1]
  location = df_personal_short.iloc[index,2]
  bedtime = df_personal_short.iloc[index,3]

  for i in range(len(df_personal_short)):

    other_name = df_personal_short.iloc[i,0]
    other_weekly_max = df_personal_short.iloc[i,1]
    other_location = df_personal_short.iloc[i,2]
    other_bedtime = df_personal_short.iloc[i,3]

  
    if int(weekly_max) <= int(other_weekly_max) and location == other_location and bedtime == other_bedtime and name!= other_name:
      print(other_name)

find_compatible()

Soline


In [ ]:
for i in range(df_personal_short):
  compatibility = find_compatible()

In [ ]:
# This block sends an email to the person who submitted the form
# email contains the return of the above function
import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = "flatmate.ucl@gmail.com"  # Enter your address
receiver_email = "enter user email here"  # Enter receiver address
password = "flatmate123"
message = "enter details about potential room mates and include email address and criteria"

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

#this part is responsible for sending email response to the user containing all potential matches for a flatmate. 

SMTPRecipientsRefused: ignored

In [ ]:
# Used few attributes per person because we have a small sample and wanted to show our idea, and how it works
# This code takes into account the most important factors (budget, location, and bedtime), not the other ones yet
